Data and codes for the copula selection results (table 3). 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import sys, os
sys.path.append('../../src/')
# import toolbox
from toolbox import *

In [2]:
# Choose copula by AIC
result_names = ['BBT_Tiingo',
                'BBT_future_Tiingo_ada', 
                'BBT_future_Tiingo_eth',
                'BBT_future_Tiingo_ltc',
                'BBT_future_Tiingo_xrp',
                'BBT_future_BITX',
                'BBT_future_BITW20', 
                'BBT_future_BITW70', 
                'BBT_future_BITW100', 
                'BBT_future_CRIX']

coin_names = ['btc', 'ada', 'eth', 'ltc', 'xrp',
              'BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']

NIG_coin_names =  ['btc', 'ada', 'eth', 'ltc', 'xrp',
              'bitx', 'bitw20', 'bitw70', 'bitw100', 'CRIX']

result_coin_mapper = dict(zip(result_names, coin_names))

In [3]:
k = {'Gaussian':1,
     't_Copula':2, 
     't_Copula_Capped':2, 
     'Clayton':1, 
     'Frank':1, 
     'Gumbel':1,  
     'Plackett':1,  
     'Gauss Mix Indep':2, 
     'rotGumbel':1, 
     'NIG':3}

copula_names = list(k.keys())

def AIC_table(l, drop_t_Copula_Capped=True):
    # l likelihood table
    if drop_t_Copula_Capped:
        l = l.drop(index='t_Copula_Capped')
       
    copula_names = list(l.index)
    AIC = l.copy()
    
    for c in copula_names:
        AIC.loc[c,:] = -2*l.loc[c,:]+2*k[c]
        
    return AIC

In [4]:
AIC_results = dict()
copula_to_use = dict()
copula_count = dict()

for result in result_names:
    l = pd.read_json('../../results/%s/MM/likelihood_NIG.json'%result) # likelihood 
#     l *=300 # 300 data points
    
    _AIC = AIC_table(l, True)
    AIC_results[result] = _AIC
    
    _copula = _AIC.idxmin(axis='rows')
    _copula = pd.DataFrame(_copula).reset_index()
    _copula.columns = ['file', 'copula']
    copula_to_use[result] = _copula
    
    copula_count[result]= pd.pivot_table(_copula, index=['copula'], aggfunc='count')

In [5]:
copula_count_summary = pd.DataFrame(columns=copula_names)

for rm in result_names:
    c = copula_count[rm]
    c.index.name = None
    c.columns = [result_coin_mapper[rm]]
    copula_count_summary = copula_count_summary.append(c.T)
    
copula_count_summary = copula_count_summary.fillna(0)

copula_count_summary = pd.concat([copula_count_summary.loc[:,c] for c in copula_names if c != 't_Copula_Capped'],
          axis=1)

copula_count_summary.loc['sum',:] = copula_count_summary.sum()

In [6]:
copula_count_percentage = []


for i in range(copula_count_summary.shape[1]):
    copula_count_percentage.append(copula_count_summary.iloc[:,i]/copula_count_summary.sum(axis=1))

In [7]:
copula_count_percentage = pd.concat(copula_count_percentage, axis=1)
copula_count_percentage.columns = copula_count_summary.columns

In [13]:
copula_count_percentage.loc[:,
                         ['t_Copula', 'Plackett',
                          'Gauss Mix Indep', 'rotGumbel', 'NIG']].iloc[:-1,:].apply(lambda x: 100*round(x,4))

,t_Copula,Plackett,Gauss Mix Indep,rotGumbel,NIG
btc,65.18,3.57,0.89,0.89,27.68
ada,0.00,0.00,0.00,0.00,100.00
eth,2.68,5.36,7.14,83.93,0.89
ltc,11.61,0.00,2.68,28.57,57.14
xrp,0.00,27.68,2.68,69.64,0.00
BITX,48.15,0.00,17.28,19.75,14.81
BITW20,0.00,0.00,0.00,96.30,3.70
BITW70,0.00,0.00,0.00,98.77,1.23
BITW100,51.85,0.00,9.88,35.80,2.47
CRIX,53.41,0.00,12.50,3.41,30.68


In [12]:
copula_count_percentage.sum(axis=1)

btc        1.0
ada        1.0
eth        1.0
ltc        1.0
xrp        1.0
BITX       1.0
BITW20     1.0
BITW70     1.0
BITW100    1.0
CRIX       1.0
sum        1.0
dtype: float64